### Full Evaluating CPM with CEBaB

This note book is based on `cebab_evaluation.ipynb`. It takes all the saved results data from running that script, and aggregate portable results. This notebook is only for sorting saved results. Nothing more.


In [1]:
import pickle
import pandas as pd

In [14]:
model_types = ["BERT", "lstm", "RoBERTa", "gpt2"]
mega_results = {}
for model in model_types:
    with open(f'../proxy_training_results/{model}-results/results.pkl', 'rb') as f:
        results_dict = pickle.load(f)
    with open(f'../proxy_training_results/{model}-control-results/results.pkl', 'rb') as f:
        control_results_dict = pickle.load(f)
    for k, v in results_dict.items():
        mega_results[k] = v
    for k, v in control_results_dict.items():
        mega_results[k] = v

In [18]:
important_keys = [
    "seed", "h_dim", "class_num", 
    "control", "beta", "gemma", 
    "cls_dropout", "enc_dropout", 
    "model_arch"
]
values = []
for k, v in mega_results.items():
    _values = []
    for ik in important_keys:
        _values.append(dict(k)[ik])
    _values.append(v[2]["ICaCE-L2"].iloc[0])
    _values.append(v[2]["ICaCE-cosine"].iloc[0])
    _values.append(v[2]["ICaCE-normdiff"].iloc[0])
    _values.append(v[-1].iloc[0][0])
    values.append(_values)
important_keys.extend(["ICaCE-L2", "ICaCE-cosine", "ICaCE-normdiff", "macro-f1"])
df = pd.DataFrame(values, columns=important_keys)
df = df.sort_values(by=['class_num'], ascending=True)

In [21]:
group_keys= [
    "class_num", 
    "control",
    "model_arch"
]
df.groupby(group_keys)[["ICaCE-L2", "ICaCE-cosine", "ICaCE-normdiff", "macro-f1"]].mean()

ICaCE-L2  ICaCE-cosine  ICaCE-normdiff  \
class_num control model_arch                                                  
2         False   bert-base-uncased   0.19742       0.77402         0.19654   
                  gpt2                0.21030       0.73714         0.20826   
                  lstm                0.25894       0.78814         0.25310   
                  roberta-base        0.19468       0.78866         0.19460   
          True    bert-base-uncased   0.29142       0.79914         0.28974   
                  gpt2                0.28668       0.75164         0.28442   
                  lstm                0.27996       0.87750         0.27278   
                  roberta-base        0.28672       0.82220         0.28656   
3         False   bert-base-uncased   0.40952       0.61886         0.33622   
                  gpt2                0.44150       0.62222         0.34850   
                  lstm                0.51288       0.65658         0.43578   
                  roberta-base        0.43158       0.63954         0.35934   
          True    bert-base-uncased   0.54710       0.73488         0.52492   
                  gpt2                0.51930       0.68638         0.47088   
                  lstm                0.52990       0.78284         0.45884   
                  roberta-base        0.54304       0.73768         0.52848   
5         False   bert-base-uncased   0.63130       0.46300         0.43188   
                  gpt2                0.54534       0.48694         0.35918   
                  lstm                0.70378       0.53916         0.53582   
                  roberta-base        0.67350       0.47492         0.48934   
          True    bert-base-uncased   0.79230       0.70180         0.70162   
                  gpt2                0.65612       0.64920         0.55032   
                  lstm                0.74136       0.72098         0.57978   
                  roberta-base        0.82404       0.63068         0.78400   

                                     macro-f1  
class_num control model_arch                   
2         False   bert-base-uncased  0.973154  
                  gpt2               0.966701  
                  lstm               0.940496  
                  roberta-base       0.976074  
          True    bert-base-uncased  0.973154  
                  gpt2               0.966701  
                  lstm               0.940496  
                  roberta-base       0.976074  
3         False   bert-base-uncased  0.818143  
                  gpt2               0.802804  
                  lstm               0.749421  
                  roberta-base       0.831138  
          True    bert-base-uncased  0.818143  
                  gpt2               0.802804  
                  lstm               0.749421  
                  roberta-base       0.831138  
5         False   bert-base-uncased  0.695808  
                  gpt2               0.653323  
                  lstm               0.596330  
                  roberta-base       0.703218  
          True    bert-base-uncased  0.695808  
                  gpt2               0.653323  
                  lstm               0.596330  
                  roberta-base       0.703218